In [1]:
pip install azure-ai-formrecognizer azure-search-documents openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.9 MB/s eta 0:00:00


In [2]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import os

In [4]:
# Set up the Form Recognizer client
endpoint = "https://.cognitiveservices.azure.com/"
api_key = ""
client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(api_key))


In [40]:
import os
os.listdir()

['.config', 'contract.pdf', 'sample_data']

In [6]:
file_path = "contract.pdf"
with open(file_path, "rb") as f:
    poller = client.begin_analyze_document("prebuilt-document", f)

In [7]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as fd:
        poller = client.begin_analyze_document("prebuilt-layout", fd)
        result = poller.result()

    # Extract text content
    extracted_text = ""
    for page in result.pages:
        for line in page.lines:
            extracted_text += line.content + "\n"
    return extracted_text

In [8]:
pdf_path = "contract.pdf"
contract_text = extract_text_from_pdf(pdf_path)
print(contract_text[:500])

Company A Contract Document
Contract Title: Supplier Agreement
This Agreement is made and entered into as of [Date], by and between:
Supplier Name: [Supplier A]
Buyer Name: [Buyer A]
Contract Duration: [Start Date] to [End Date]
1. Rebates/Volume Discounts
The Supplier agrees to provide volume-based discounts to the Buyer, incentivizing higher
purchase volumes with proportional rebates. For each calendar quarter, the Supplier will
assess the total purchase volume made by the Buyer and apply tier


In [9]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


In [11]:
search_endpoint = "https://.search.windows.net"
search_api_key = ""
index_name = "contind"
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=AzureKeyCredential(search_api_key))


In [12]:
def create_or_update_index(text):
    # Create a simple index for storing clause titles and content
    documents = [
        {
            "id": "contract1",  # Unique document ID
            "title": "Contract Title",  # Optionally, extract or add title
            "text": text  # Contract text or specific clauses
        }
    ]
    # Upload the document
    search_client.upload_documents(documents=documents)

# Index the extracted contract text
create_or_update_index(contract_text)

In [14]:
def search_for_clause(clause_title):
    results = search_client.search(clause_title)  # Search for a specific clause title
    clauses = []
    for result in results:
        clauses.append(result['text'])
    return clauses

# Search for a specific clause
clause_title = "Inflation Adjustment"
clauses = search_for_clause(clause_title)
print(clauses)

["Company A Contract Document\nContract Title: Supplier Agreement\nThis Agreement is made and entered into as of [Date], by and between:\nSupplier Name: [Supplier A]\nBuyer Name: [Buyer A]\nContract Duration: [Start Date] to [End Date]\n1. Rebates/Volume Discounts\nThe Supplier agrees to provide volume-based discounts to the Buyer, incentivizing higher\npurchase volumes with proportional rebates. For each calendar quarter, the Supplier will\nassess the total purchase volume made by the Buyer and apply tiered discounts once certain\nthresholds are met. Volume thresholds and their associated rebate percentages are detailed\nin Appendix A of this contract. Should the Buyer's purchases meet or exceed a specified\nthreshold within the quarter, a corresponding rebate will be issued in the form of a credit\nnote, applied to the following quarter's orders.\nIf the Buyer reaches multiple thresholds within the same quarter, the Supplier will apply the\nhighest qualifying rebate for that period. 

In [15]:
!pip install openai

In [16]:
import openai
import os

In [32]:
from openai import AzureOpenAI
client = AzureOpenAI(
  api_key="3BwtD0zKmB0buQT43BeyhBonEL3Sh0FaHuOplzTlvscsIQuhHHXgJQQJ99AKAC5RqLJXJ3w3AAABACOGOJ5B",
  api_version = "2024-02-01",
  azure_endpoint = "https://arzoo-m3bvvhbw-westeurope.openai.azure.com/"
)


In [35]:

def summarize_clause(clause_text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant specialized in summarizing legal clauses."},
        {"role": "user", "content": f"Please summarize the following clause: {clause_text}"}
    ]

    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )
    summary = response.choices[0].message.content.strip()
    return summary

In [36]:
# Summarize the found clauses
summary = summarize_clause(clauses[0])
'''
for clause in clauses:
    summary = summarize_clause(clause)
    print(f"Summary: {summary}")
'''

In [39]:
# List of clause titles you are looking for
'''
clause_titles = [
    "Termination Provisions",
    "Confidentiality",
    "Indemnification",
    "Rebates/Volume Discounts",
    "Payment Terms",
    "SLAs/Service Credits",
    "Price Protection",
    "Forex",
    "COLA/Inflation Adjustment",
    "Late Payment Penalties"
]
'''
clause_titles = [
    "Rebates/Volume Discounts",
    "Payment Terms",
    "Invalid"
]

def generate_report(clause_titles):
    report = ""
    for title in clause_titles:
        clauses = search_for_clause(title)
        if clauses:
            report += f"Clause: {title}\n"
            #for clause in clauses:
            summary = summarize_clause(clauses[0])
            report += f"Summary: {summary}\n\n"
        else:
            report += f"Clause: {title} - Absent\n\n"
    return report

# Generate and print the final report
final_report = generate_report(clause_titles)
print(final_report)


Clause: Rebates/Volume Discounts
Summary: This is the Supplier Agreement contract between Supplier A and Buyer A, effective from the start date to the end date. The agreement contains clauses related to rebates/volume discounts, SLAs/service credits, price protection, forex, COLA/inflation adjustment, payment terms, early payment discounts, late payment penalties, termination provisions, committal/non-committal contracts, force majeure, confidentiality, intellectual property, dispute resolution, governing law, non-compete, indemnification, audit rights, and performance bonds. Any amendments or modifications to this contract must be in writing and signed by authorized representatives of both parties.

Clause: Payment Terms
Summary: This clause outlines the terms of the Supplier Agreement between Supplier A and Buyer A. The agreement includes provisions for volume-based discounts, service level agreements, pricing protection, forex, inflation adjustments, payment terms, termination, and 